In [148]:
import os, errno
from os import path
import requests
from bs4 import BeautifulSoup as bs
import regex as re
import copy
import pandas as pd
from urlparse import urlparse
from datetime import datetime


In [168]:
mol_diag_url = 'https://www.genomeweb.com/molecular-diagnostics'
r = requests.get(mol_diag_url)

In [169]:
r.status_code

200

In [170]:
r.content;

In [171]:
soup = bs(r.content, "lxml")

In [175]:
print(soup.prettify());

<!DOCTYPE html>
<html dir="ltr" lang="en" xmlns:article="http://ogp.me/ns/article#" xmlns:book="http://ogp.me/ns/book#" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:product="http://ogp.me/ns/product#" xmlns:profile="http://ogp.me/ns/profile#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:schema="http://schema.org/" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:video="http://ogp.me/ns/video#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
 <head profile="http://www.w3.org/1999/xhtml/vocab">
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t]

In [191]:
div = soup.find("div", {"class": "clearfix fieldlayout-region-top"})
#div = soup.find("div", {"class": "node-title fieldlayout node-field-title"})  
print div

<div class="clearfix fieldlayout-region-top"><h3 class=" node-title fieldlayout node-field-title"><a href="/business-news/startup-spindiag-raises-16m-seed-funding">Startup SpinDiag Raises €1.6M in Seed Funding</a></h3></div>


In [192]:
div.find('a')['href']

'/business-news/startup-spindiag-raises-16m-seed-funding'

In [16]:
print soup.title

<title>Molecular Diagnostics News | GenomeWeb</title>


In [5]:
def get_pages(base_url='https://www.genomeweb.com',
            subsection = '/molecular-diagnostics'):
    ''' This function takes a particular url and subsection from genomeweb as input 
    and returns a list with the complete set of pages following the next button '''    
    
    # Define url
    url = base_url+subsection
    page_list = []
    page_list.append(url)
    r = requests.get(url)
    soup = bs(r.content)
    
    #This checks if we start on first page - enumerated differently from the rest in genomeweb
    if soup.find("div", {"class": "more-link"}):
        
        # finds division for class more link
        div = soup.find("div", {"class": "more-link"})  
        
        #extract the hyperlink from the division 
        elm = div.find('a')['href']
        
        # makes a link for next page based on base url and hiperlink 
        next_page_link = base_url + elm
        page_list.append(next_page_link)
        
        # extract data from second page in genomeweb - first enumerated as the rest 
        r = requests.get(next_page_link)
        soup = bs(r.content)
        page_list.append(next_page_link)
    else:    
        next_page_link = url
    
    # This loop will start with the second page or higher and immediatly move to next one
    # until last page has been listed
    while soup.find('a', {'title': 'Go to next page'}) :
        print next_page_link
        
        # extract hyperlink for next page
        elm = soup.find('a', {'title': 'Go to next page'})['href']
        next_page_link = base_url + elm
        r=r = requests.get(next_page_link)
        soup = bs(r.content)
        page_list.append(next_page_link)
    
    return page_list

In [6]:
page_list = get_pages('https://www.genomeweb.com','/channel/33?page=270')

/Users/ale/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/ale/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


https://www.genomeweb.com/channel/33?page=270
https://www.genomeweb.com/channel/33?page=271
https://www.genomeweb.com/channel/33?page=272
https://www.genomeweb.com/channel/33?page=273


In [285]:
def scrape_page(page_list):
    regex = r'\>(.*?)\<'
    columns = ['Title','Hyperlink', 'Info', 'Date']
    df = pd.DataFrame(columns = columns)
    for page in page_list:
        parsed_uri = urlparse(page)
        base_url = '{uri.scheme}://{uri.netloc}'.format(uri=parsed_uri)
        print 'information extrcated from: ',base_url
        html = requests.get(page)
        soup = bs(html.content, "lxml")
        div = soup.find("div", {"class": "view-content"})
        a = div.find_all('a')
        p = div.find_all('p')
        d = soup.find_all('span',{'class':'date-display-single'})
        for link,doc,info in zip(a,d,p):
            print link
            hiperlink = str(base_url+link['href'])
            match = re.search(regex, str(link))
            #print len(match), match
            if (max(match.span(0))-min(match.span(0)))>(max(match.span(1))-min(match.span(1))):
                title = match[0][1:-1]
            else:
                title = match[1][1:-1]
            print title
                    
            
            date = doc['content']
            date = date[:10]
            info = str(info)[3:-4]
            df = df.append({'Hyperlink': hiperlink, 'Title':title, 'Date': date, 'Info':info}, ignore_index=True)  
        #for doc in d:
        #    date = doc['content']
        #    date = date[:10]
        #    df = df.append({'Date':date}, ignore_index=True)
    
        return df
    


#scrape_page(['https://www.genomeweb.com/channel/33?page=273'])
df = scrape_page(['https://www.genomeweb.com/channel/33?page=1'])

information extrcated from:  https://www.genomeweb.com
<a href="/business-news/oncodna-inks-morocco-distribution-deal">OncoDNA Inks Morocco Distribution Deal</a>
OncoDNA Inks Morocco Distribution Deal
<a href="/molecular-diagnostics/quanterix-partners-develop-pcr-free-mirna-detection-method-liver-toxicity">Quanterix, Partners Develop PCR-Free miRNA Detection Method for Liver Toxicity Testing</a>
Quanterix, Partners Develop PCR-Free miRNA Detection Method for Liver Toxicity Testing
<a href="/regulatory-news/t2-biosystems-receives-ce-mark-bacterial-panel">T2 Biosystems Receives CE Mark for Bacterial Panel</a>
T2 Biosystems Receives CE Mark for Bacterial Panel
<a href="/molecular-diagnostics/castle-biosciences-uveal-melanoma-prognostic-test-garners-coverage-multiple">Castle Biosciences Uveal Melanoma Prognostic Test Garners Coverage from Multiple Payors</a>
Castle Biosciences Uveal Melanoma Prognostic Test Garners Coverage from Multiple Payors
<a href="/molecular-diagnostics/courtagen-clo

In [286]:
df
#df.iloc[7,0]

,Title,Hyperlink,Info,Date
0,OncoDNA Inks Morocco Distribution Deal,https://www.genomeweb.com/business-news/oncodn...,Diagnostics firm BioHisto Maroc will market an...,2017-07-18
1,"Quanterix, Partners Develop PCR-Free miRNA Det...",https://www.genomeweb.com/molecular-diagnostic...,They demonstrated the applicability of the Sim...,2017-07-18
2,T2 Biosystems Receives CE Mark for Bacterial P...,https://www.genomeweb.com/regulatory-news/t2-b...,T2 said that the panel will be commercially av...,2017-07-18
3,Castle Biosciences Uveal Melanoma Prognostic T...,https://www.genomeweb.com/molecular-diagnostic...,The company announced positive coverage decisi...,2017-07-18
4,"Courtagen Closes Genetic Testing Business, Cit...",https://www.genomeweb.com/molecular-diagnostic...,"Going forward, the company will focus on its w...",2017-07-18
5,"Agena, HeartGenetics Ink Genetic Testing Comar...",https://www.genomeweb.com/molecular-diagnostic...,The companies will market Agena's MassArray Dx...,2017-07-18
6,New Study Bolsters View That Mosaic Embryos Fr...,https://www.genomeweb.com/molecular-diagnostic...,"At a conference last week, researchers from It...",2017-07-14
7,,https://www.genomeweb.com/molecular-diagnostic...,"In a revised medical policy, Aetna said it wou...",2017-07-14
8,Aetna Won't Cover Noninvasive Prenatal Testing...,https://www.genomeweb.com/molecular-diagnostic...,The team showed in a study this week that they...,2017-07-14
9,Nanopore Researchers Developing Mutation Detec...,https://www.genomeweb.com/molecular-diagnostic...,Recently launched tests that screen for single...,2017-07-14
